In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from keras.models import load_model
from keras.applications.resnet50 import preprocess_input as preprocess_input_resnet50
from keras.applications.xception import preprocess_input as preprocess_input_xception

Using TensorFlow backend.


Read in the data.

In [2]:
test_set = pd.read_csv('test_set_metadata.csv')
X_test = np.load('test_set_hmgd_arr.npy')

model_G1 = load_model('model_custom_small_epoch_019.hdf5')
model_G2 = load_model('model_ResNet50_finetuning_epoch_011.hdf5')
model_G3 = load_model('model_Xception_finetuning_epoch_005.hdf5')
model_G4 = load_model('model_custom_big_epoch_034.hdf5')

Compute predictions.

In [3]:
model_G1_preds = model_G1.predict_proba(255 * X_test)
model_G2_preds = model_G2.predict(preprocess_input_resnet50(255 * X_test))
model_G3_preds = model_G3.predict(preprocess_input_xception(255 * X_test))
model_G4_preds = model_G4.predict_proba(255 * X_test)
ensemble_preds = (model_G1_preds + model_G2_preds + model_G3_preds + model_G4_preds) / 4

# free up memory
del X_test

Compute model ensemble accuracy.

In [5]:
ensemble_accuracy = ((ensemble_preds.ravel() > 0.5).astype(np.int) == test_set['gender']).mean()
print(f'Model ensemble accuracy: {ensemble_accuracy}')

Model ensemble accuracy: 0.8994882561343656
